In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def calculateHistogram(img, title, saveName):
  colors = ('b','g','r')
  for i,color in enumerate(colors):
    hist = cv2.calcHist([img],[i],None,[256],[0,256])
    plt.plot(hist,color = color)
  plt.title(title)
  plt.savefig(saveName)
  plt.show()

def NCC(img1, img2):
  img1 = img1 - img1.mean(axis = 0)
  img2 = img2 - img2.mean(axis = 0)
  normImg1 = img1 / np.linalg.norm(img1)
  normImg2 = img2 / np.linalg.norm(img2)
  return np.sum(normImg1 * normImg2)

def meanStdev(img):
  mean, std = cv2.meanStdDev(img)
  mean = np.array([mean[0][0], mean[1][0], mean[2][0]])
  std = np.array([std[0][0], std[1][0], std[2][0]])
  return mean, std

def colorTransfer(sourceIn, targetIn, startRow, startCol, endRow, endCol):
  # 1. Convert the source and target images to lαβ color space representation from RGB color space.

  source = cv2.cvtColor(sourceIn, cv2.COLOR_BGR2LAB)
  target = cv2.cvtColor(targetIn, cv2.COLOR_BGR2LAB)

  # Convert to float32
  source = np.float32(source) 
  target = np.float32(target)

  # 2. For each channel of source and target images,
  # compute means (µl, µα, µβ) 
  # and standard deviations (σl, σα, σβ).

  source_mean, source_std = meanStdev(source)
  target_mean, target_std = meanStdev(target)
  

  resultImage = np.copy(source)

  for row in range(startRow, endRow): # source.shape[0]
    for col in range(startCol, endCol): # source.shape[1]
      for chl in range(3): # source.shape[2]
        # 3. Subtract means from the data points for source image.
        value = source[row][col][chl]
        temp = value - source_mean[chl]


        # 4. Scale new data points according to the relative standard deviations of target and source images.

        stdev_ratio = (target_std[chl] / source_std[chl])
        newValue = (temp * stdev_ratio)


        #5. Add the averages computed for the target to scaled data points

        newValue = newValue + target_mean[chl]

        # Round newValue
        newValue = round(newValue)

        # Clip newValue in range(0, 255)
        if newValue < 0:
          newValue = 0
        if newValue > 255:
          newValue = 255

        # Save newValue to resultImage
        resultImage[row][col][chl] = newValue

  # Convert back from float32
  resultImage = np.uint8(resultImage)

  # Convert back from LAB to RGB
  resultImage = cv2.cvtColor(resultImage, cv2.COLOR_LAB2BGR)

  return resultImage

def part2(sourceIn, targetIn, rowSlice, colSlice):
  sRow, sCol, sChl = sourceIn.shape

  rowCount = round(sRow / rowSlice) # Divide into "rowSlice" sized equal parts
  colCount = round(sCol / colSlice) # Divide into "colSlice" sized equal parts

  nccValues = np.empty([rowCount, colCount]) # To keep all NCC values for all equal parts

  for i in range(0, sRow - rowSlice, rowSlice):
    for j in range(0, sCol - colSlice, colSlice):

      ncc1 = sourceIn[i : i + rowSlice, j : j + rowSlice] # Get one equal part from source
      ncc2 = targetIn[i : i + colSlice, j : j + colSlice] # Get one equal part from target

      try:
          ncc = NCC(ncc1, ncc2) # Calculate NCC for equal parts
          tempI, tempJ = int(i / rowSlice), int(j / colSlice)
          nccValues[tempI][tempJ] = ncc # Put NCC value into array
      except ValueError:
        pass
      

  sortedNcc = np.flip(np.sort(nccValues, axis = None)) # Sort NCC array with descending order

  count = 0
  for x in sortedNcc:
    if x > 0: # If similarity is high, this part is going to get color transfer
      count += 1
  
  resultImage = np.copy(sourceIn) # Copy the source image

  for k in range(count):
    if np.isnan(sortedNcc[k]):
      continue
    # Find the exact location of the region to be color transferred
    r, c = np.where(nccValues == sortedNcc[k])
    startRow = r[0] * rowSlice
    startCol = c[0] * colSlice
    endRow = (r[0] + 1) * rowSlice
    endCol = (c[0] + 1) * colSlice

    # Clip for max index
    if endRow >= resultImage.shape[0]:
      endRow = resultImage.shape[0] - 1
    if endCol >= resultImage.shape[1]:
      endCol = resultImage.shape[1] - 1

    # Do color transfer to this region
    resultImage = colorTransfer(resultImage, targetIn, startRow, startCol, endRow, endCol)
  
  return resultImage

# Main
# Read source and target image.

img1 = cv2.imread("/content/sample_data/in_01.png")
img2 = cv2.imread("/content/sample_data/tar_01.png")

finalImage = colorTransfer(img1, img2, 0, 0, img1.shape[0], img1.shape[1]) # Call colorTransfer function for part1

# IF YOU WANT TO RUN PART2, UNCOMMENT THIS
#part2Image = part2(img1, img2, 10, 10) # Call part2
# rowSlice and colSlice in part2 is optional, you can change these to change rectangular regions' size

cv2_imshow(finalImage)
#cv2_imshow(part2Image)

cv2.imwrite("/content/sample_data/myresult01.png", finalImage) # To Save the Image

calculateHistogram(img1, "Histogram for Input Image", "/content/sample_data/in_hist01.png") # To Create and Save the Input Image Histogram
calculateHistogram(finalImage, "Histogram for Final Image", "/content/sample_data/res_hist01.png") # To Create and Save the Result Image Histogram